# Working with medical images
---
## Part 1 - the basics
Medical images are much more than multi-dimensional arrays.

What does an image have that an array does not?
- *Spatial information*
- *Metadata*
- *Photometric interpretation*

Let's go through some of these concepts and talk about why they are important for image processing and analysis.

This notebook only goes through spatial information, so let's come back to that.

#### Metadata

This actually includes the spatial information, but also other kinds of information relevant to the image:
 - pixel depth
 - type of scanner
 - scan parameters (e.g. MRI sequence, PET tracer)
 - scan duration (relevant for nuclear medicine)
 - patient information
 
#### Photometric interpretation

This tells us how we should display the image using either gray scale (MONOCHROME/MONOCHROME2) or a specific color map (pseudo-colour, or true color).

In [ ]:
# Only run this if you don't have platipy
#!pip install git+https://github.com/pyplati/platipy

In [ ]:
# !dev 
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

from platipy.imaging.visualisation.tools import ImageVisualiser

%matplotlib notebook

To start with, we will use some dummy data generated by inserting spheres into an array

In [ ]:
# To start off easy we wi

def insert_sphere(arr, sp_radius=4, sp_centre=(0,0,0)):
    
    arr_copy = arr[:]
    
    x, y, z = np.indices(arr.shape)
    
    arr_copy[ (x-sp_centre[0])**2 +
              (y-sp_centre[1])**2 +
              (z-sp_centre[2])**2 <=
              sp_radius**2 ] = 1
    
    return arr_copy

Now we make an array and insert a big (radius=30) and small (radius=15) sphere

They have the same element for the first and third axis.
Usually we store numpy image arrays with:
 - Index 0 = axial (superior - inferior) = z-axis
 - Index 1 = coronal (posterior - anterior) = y-axis
 - Index 2 = sagittal (left - right) = x-axis
 
 This makes it easier to consistently convert between **numpy** arrays (np.ndarray) and **SimpleITK** images (sitk.Image).

In [ ]:
# Create array with 100 axial slices, 200 coronal slices, 300 sagittal slices
arr = np.zeros((100,200,300))
# Insert big sphere
arr_sphere = insert_sphere(arr, 30, (50,100,150))
# Insert little sphere
arr_sphere = insert_sphere(arr, 15, (50,150,150))

It's pretty common to use **matplotlib** to plot images. Later on we will see why this can be a bit misleading.

For now, we will plot slices taken at orthogonal angles and passing through the centre of the large sphere.

In [ ]:
# Set up the figure and axes
# We can adjust the axes width and height ratios to match the array size
# Top left = axial slice (z-axis)
# Bottom left = coronal slice (y-axis)
# Bottom right = sagittal (x-axis)
fig, axes = plt.subplots(2,2, figsize=(6,4),
                         gridspec_kw={
                             "width_ratios":(arr.shape[2],arr.shape[1]),
                             "height_ratios":(arr.shape[1],arr.shape[0]),
                         })

# Plot through the centre as defined earlier
axes[0,0].imshow(arr_sphere[50,:,:])
axes[1,0].imshow(arr_sphere[:,100,:])
axes[1,1].imshow(arr_sphere[:,:,150])

# Turn of the top right (empy) plot
axes[0,1].axis('off')

# Labels - if you want
axes[0,0].set_title('x-axis')
axes[1,0].set_title('y-axis')
axes[1,1].set_title('z-axis')

# Set to remove unnecaary space around axes
fig.tight_layout()

# Show - can also save to disk
fig.show()
fig.savefig("./figures/figure_1_spheres_original.png", dpi=300, transparent=True)

Now, let's build an image from our array.
All of the spatial information is set to the default values:
- Spacing: 1.0mm × 1.0mm × 1.0mm
- Origin: (0,0,0)
- Direction: $$\begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$


In [ ]:
# Generate image
img_sphere = sitk.GetImageFromArray(arr_sphere)
print("Array info: ", arr_sphere.shape, arr_sphere.dtype)
# itk::Image< TPixel, VImageDimension >
print("Image info: ", img_sphere)

In [ ]:
# Write the image to open in Slicer

sitk.WriteImage(img_sphere, "./output/spheres_original.nii.gz")

Since this type of plotting is pretty excessive we are going to use platipy to do the hard work for us

In [ ]:
vis = ImageVisualiser(img_sphere, window=[0,1], figure_size_in=4)
fig = vis.show(interact=True)
# You can save the figure like before
fig.savefig("./figures/figure_2_spheres_original_platipy.png", dpi=300, transparent=True)

Now we will change some image spatial information. 

In this example the origin, which we move by a shift of -10 along the first axis.

##### IMPORTANT
When we change variables in SimpleITK the axes are the reverse of numpy arrays.
So the first axis is the sagittal direction (x-axis).

In [ ]:
# Create a modified image.
img_modified = sitk.GetImageFromArray(arr_sphere)
# Change the origin
img_modified.SetOrigin((-10, 0, 0))
# Save for Slicer
sitk.WriteImage(img_modified, "./output/spheres_modified_origin.nii.gz")

In [ ]:
# We can generate an array similar to how we generate images
arr_modified = sitk.GetArrayViewFromImage(img_modified)

In [ ]:
# We can check that the array is the same
np.all(arr_modified==arr_sphere)

In [ ]:
print("Original arr_sphere: ",i_0:=hex(id(arr_sphere)))
print("Modified arr_modified: ",i_1:=hex(id(arr_modified)))
print("The same? ", i_0==i_1)

Next, we change the direction matrix to:
$$\begin{bmatrix} 0 & 1 & 0 \\ 1 & 0 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$
This means the first and second axes are swapped.

In this new image, the first axis will be the coronal direction and the second axis is the sagittal direction.

In [ ]:
# Another image
img_modified = sitk.GetImageFromArray(arr_sphere)
# Change the direction - the matrix is entered as an iterable 
img_modified.SetDirection((0,1,0,1,0,0,0,0,1))
# Save the image
sitk.WriteImage(img_modified, "./output/spheres_modified_direction.nii.gz")

In [ ]:
# The image looks exactly the same
# Spatial information lost!
# All figures in platipy are displayed using the identity matrix as the direction matrix
# BE CAREFUL!
vis = ImageVisualiser(img_modified, window=(0,1), figure_size_in=6)
fig = vis.show()

Last example - let's change the image spacing to:

$$\left(0.98\text{ mm},\ 0.98\text{ mm},\ 3.00\text{ mm}\right)$$ 

This is pretty typical image spacing for CT scans. 

In [ ]:
# Another image
img_modified = sitk.GetImageFromArray(arr_sphere)
# Change the spacing
img_modified.SetSpacing((0.98,0.98,3))

# Visualsie
vis = ImageVisualiser(img_modified, window=(0,1), figure_size_in=6)
fig = vis.show()

# Save the image
sitk.WriteImage(img_modified, "./output/spheres_modified_spacing.nii.gz")

## Resampling

Resampling is a **super** important technique for medical image processing and analysis.

In a nutshell, the image is *moved* into the reference image space.

The voxels (= array data) are calculated using interpolation.

In [ ]:
# Resample the image spacing of the modified to return it to the original
img_modified_res = sitk.Resample(img_modified, img_sphere, interpolator=sitk.sitkNearestNeighbor)

In [ ]:
# Save the image
sitk.WriteImage(img_modified_res, "./output/spheres_resampled.nii.gz")

In [ ]:
# Platipy can overlay images
vis = ImageVisualiser(img_sphere, window=[0,1], figure_size_in=6)
vis.add_comparison_overlay(img_modified_res)
fig = vis.show()

Now we can see why all this stuff matters. If we change image spatial information

In [ ]:
# Before we changed the image spatial information the values in each voxel are the same
np.all(
    sitk.GetArrayFromImage(img_modified)==sitk.GetArrayFromImage(img_sphere)
)

In [ ]:
# This is not true (obviously) if we change the spatial information and resample.
# We would also need to scale the image down...
np.all(
    sitk.GetArrayFromImage(img_modified_res)==sitk.GetArrayFromImage(img_sphere)
)

## "Real" images

To end this notebook, let's just check out a few real images.

In [ ]:
# A H&N CT scan
img_hn_ct = sitk.ReadImage("./input/HN_CT.nii.gz")

# Thanks again platipy
vis = ImageVisualiser(img_hn_ct, cut=(70,256,256), figure_size_in=5)
fig = vis.show()

print(img_hn_ct)

In [ ]:
# A H&N PT (positron tomography) scan
img_hn_pt = sitk.ReadImage("./input/HN_PT.nii.gz")

vis = ImageVisualiser(
    img_hn_pt,
    cut=(70,64,64),
    window=(0,50000),
    colormap=plt.cm.magma,
    figure_size_in=5)

fig = vis.show()

print(img_hn_pt)

In [ ]:
# A little bit of resampling
img_hn_pt_res = sitk.Resample(img_hn_pt, img_hn_ct)

# Overlay the images
vis = ImageVisualiser(
    img_hn_ct,
    cut=(70,256,256),
    figure_size_in=5
)

vis.add_scalar_overlay(
    img_hn_pt_res,
    name='PET voxel values',
    colormap=plt.cm.magma,
    max_value=50000
) # Wowsa

fig = vis.show()

In [ ]:
# A prostate MRI
img_prostate = sitk.ReadImage("./input/PROSATE_MR.nii.gz")
original_dir = img_prostate.GetDirection()

vis = ImageVisualiser(img_prostate, window=(0,1500), figure_size_in=5)
fig = vis.show()

print(img_prostate)

In [ ]:
# A prostate MRI
img_prostate.SetDirection((1,0,0,0,1,0,0,0,1))

vis = ImageVisualiser(img_prostate, window=(0,1500), figure_size_in=5)
fig = vis.show()

print(img_prostate)

In [ ]:
print("Image origin:             ", img_prostate.GetOrigin())
print("Transformed array origin: ", img_prostate.TransformIndexToPhysicalPoint((0,0,0)))

In [ ]:
# Change the direction back to the original
img_prostate.SetDirection(original_dir)

img_prostate_res = sitk.Resample(
    img_prostate,
    size = (400,400,50),
    transform = sitk.Transform(),
    interpolator = sitk.sitkBSpline,
    outputOrigin = (-80, -40, -120), # Hard to find these automatically
    outputSpacing = img_prostate.GetSpacing(),
    outputDirection = (1,0,0,0,1,0,0,0,1), # Change the direction in this step
    defaultPixelValue = 0
)

sitk.WriteImage(img_prostate_res, "./output/prostate_mr_resampled.nii.gz")

vis = ImageVisualiser(img_prostate_res, window=(0,1500), figure_size_in=5)
fig = vis.show()

We have literally changed the voxel values (=array data) by rotating the image volume in space. 

After this we sample (hence the name) points on a three-dimensional grid. 

The direction of the axes defining that grid are given by the direction matrix.

We also moved the origin of the grid - and hence shifted the image over.

### WAIT A MINUTE...

If we can rotate and shift images in space aren't we basically doing registration?

We will do just this in the next notebook, but first...

In [ ]:
# Medical imaging can also include photographs

img_surprise = sitk.ReadImage("./input/SURPRISE.png")

print(img_surprise)

In [ ]:
# Platipy doesn't support photographs yet!

arr_surprise = sitk.GetArrayViewFromImage(img_surprise)

fig, ax = plt.subplots(1,1,figsize = (5, 5/np.divide(*img_surprise.GetSize())))

ax.imshow(arr_surprise)

fig.tight_layout()

fig.show()